In [1]:
import openai
import gradio as gr
import pandas as pd
from datetime import datetime
import asyncio
from dotenv import load_dotenv

import os
import logging
import sys
import json

In [2]:
load_dotenv(override=True)

True

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Ensure the API keys are loaded correctly
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it in your .env file.")
else:
    openai.api_key = openai_api_key

if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY environment variable not set. Please set it in your .env file.")
else:
    # Return both API keys
    (openai_api_key, gemini_api_key)


In [4]:
# Tools: Simulate Order Status Lookup Tool
# ----------------------------------------
def tool_get_order_status(order_id):
    if not order_id:
        return "Order ID is required to fetch the order status."
    return f"Order {order_id} was shipped on May 13 and is expected to arrive by May 17."

In [5]:
# Guardrails Tool: Enforce Policy Compliance
# ----------------------------------------
def tool_enforce_guardrails(message):
    blocked_keywords = ["refund after 30 days", "free for all", "unlimited returns"]
    for keyword in blocked_keywords:
        if keyword.lower() in message.lower():
            return f"⚠️ Guardrail Alert: Message contains restricted phrase '{keyword}'. Please revise."
    return ""

In [6]:
# Professional System Prompt
# ----------------------------
SYSTEM_PROMPT = """
You are an AI-powered multilingual customer support assistant working for a premium ecommerce brand.
Your mission is to help customers resolve their inquiries effectively, professionally, and empathetically.

Always adhere to the following rules:
1. Answer in the language used by the customer (English, French, Spanish, Portuguese, or German).
2. Keep your tone polite, friendly, and brand-consistent (e.g., professional, warm, and clear).
3. Do NOT promise anything beyond the provided policies.
4. Only use tools or resources you are provided with.
5. If you do not have the answer, ask the customer for clarification.

### Company Policy:
Return Policy: "We accept returns within 30 days of purchase if items are unused and in original packaging."
Shipping Policy: "Orders are typically shipped within 2 business days and delivered within 5-7 days."

Ensure your responses are accurate, helpful, and policy-compliant. Avoid speculative or misleading information.
"""

In [7]:
# Assistant memory (in-session only)
# ----------------------------
chat_history = []

In [15]:
# Assistant Completion Function with Async and Guardrails
# ----------------------------
def ask_openai_agent(user_message, language, order_id=None):
    global chat_history

    tool_context = tool_get_order_status(order_id) if order_id else ""
    guardrail_warning = tool_enforce_guardrails(user_message)

    if guardrail_warning:
        return guardrail_warning

    user_prompt = f"Customer message (Language: {language}): {user_message}\n{tool_context}"

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
    ] + chat_history + [
        {"role": "user", "content": user_prompt}
    ]

    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0.4,
            max_tokens=512
        )
        reply = response.choices[0].message.content
        chat_history.append({"role": "user", "content": user_prompt})
        chat_history.append({"role": "assistant", "content": reply})
        return reply
    except Exception as e:
        return f"[Error from OpenAI API]: {str(e)}"


In [16]:
# Summarize the conversation
# ----------------------------
def summarize_conversation():
    if not chat_history:
        return "No conversation to summarize."
    summary_prompt = [
        {"role": "system", "content": "Summarize the following customer support conversation in a professional tone."},
        *chat_history
    ]
    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=summary_prompt
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"[Error summarizing conversation]: {str(e)}"

In [17]:
# Gradio UI
# ----------------------------
def respond_to_customer(message, language, order_id):
    return ask_openai_agent(message, language, order_id)

In [18]:
def clear_chat():
    global chat_history
    chat_history = []
    return "Chat memory has been cleared."

In [19]:
# Initialize log_data as an empty list if not already defined
log_data = []

def download_log():
    df = pd.DataFrame(log_data)
    return df.to_csv(index=False)

In [20]:
with gr.Blocks() as demo:
    gr.Markdown("""
    # 🛍️ Multilingual AI Customer Support Assistant
    - Uses OpenAI GPT-4 with context-aware prompting and simulated tools.
    - Responds to customer queries in multiple languages (EN, FR, ES, PT, DE).
    - Enforces brand tone, policies, and safety guardrails as callable tools.
    - Includes feedback logging and conversation summarization.
    """)

    with gr.Row():
        message = gr.Textbox(label="Customer Message", placeholder="Enter customer inquiry...", lines=2)
        language = gr.Dropdown(["English", "French", "Spanish", "Portuguese", "German"], value="English", label="Language")
        order_id = gr.Textbox(label="Order ID (optional)")

    output = gr.Textbox(label="AI Assistant Response", lines=3)
    feedback = gr.Radio(["👍 Helpful", "👎 Unhelpful"], label="Was this response helpful?")
    notes = gr.Textbox(label="Additional Feedback", placeholder="Your thoughts...")

    with gr.Row():
        submit = gr.Button("Send")
        clear = gr.Button("Clear Memory")
        summarize = gr.Button("Summarize Chat")
        download = gr.Button("📥 Export Log")

    submit.click(fn=respond_to_customer, inputs=[message, language, order_id], outputs=output)
    clear.click(fn=clear_chat, outputs=output)
    summarize.click(fn=summarize_conversation, outputs=output)
    download.click(fn=download_log, outputs=gr.File(label="Download CSV"))

In [21]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/linoospaulinos/miniforge3/envs/py-learn-py310/lib/python3.10/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "/Users/linoospaulinos/miniforge3/envs/py-learn-py310/lib/python3.10/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/linoospaulinos/miniforge3/envs/py-learn-py310/lib/python3.10/site-packages/gradio/blocks.py", line 2054, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/Users/linoospaulinos/miniforge3/envs/py-learn-py310/lib/python3.10/site-packages/gradio/blocks.py", line 1860, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/Users/linoospaulinos/miniforge3/envs/py-learn-py310/lib/python3.10/site-packages/gradio/components/file.py", line 223, in postprocess
    size=Path(value).stat()